<a href="https://colab.research.google.com/github/Fogeinator/foodrescue/blob/master/foodrescueMain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Getting the Dataset

https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification

In [ ]:
!pip install --quiet kaggle

In [ ]:
# Choose kaggle.json that created for new API token in your account
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Check the file in its new directory
!ls /root/.kaggle/
# Check the file permission
!ls -l ~/.kaggle/kaggle.json
# Change the file permission
# chmod 600 file – owner can read and write | chmod 700 file – owner can read, write and execute
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Get the dataset we want
# CAUTION: IT IS 4 MF GB OF DATA
!kaggle datasets download -d sriramr/fruits-fresh-and-rotten-for-classification

In [ ]:
# Unzip the dataset
!mkdir data
!unzip fruits-fresh-and-rotten-for-classification.zip -d data

## Training the Model

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [ ]:
data_dir = 'data/dataset/train'

# split train test data for training
def load_split_train_test(datadir, valid_size = .2):
    # transform images
    train_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                       transforms.ToTensor(),
                                       ])
    test_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                      transforms.ToTensor(),
                                      ])
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)

    from torch.utils.data.sampler import SubsetRandomSampler

    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=64)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=64)
    
    return trainloader, testloader

    
trainloader, testloader = load_split_train_test(data_dir, .2)
print(trainloader.dataset.classes)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
# print(model)

In [ ]:
for param in model.parameters():
    param.requires_grad = False
    
# fully connected layer in NN
model.fc = nn.Sequential(nn.Linear(2048, 512),
                         nn.ReLU(),
                         nn.Dropout(0.2),
                         nn.Linear(512, 10),
                         nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
model.to(device)


In [ ]:
# actual model training
# note: use GPU or else you will die of waiting with CPU

# 2 is more than enough: see why in the graph below
epochs = 2
steps = 0
running_loss = 0
print_every = 10
train_losses, test_losses = [], []

for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)

                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()
torch.save(model, 'fruitmodel.pth')


In [ ]:
# plot of training and testing losses
matplotlib.rc('xtick',color='w')
matplotlib.rc('ytick',color='w')

plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=True)

plt.show()

## Using the Model

In [ ]:
data_dir = 'data/dataset/test'
test_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                      transforms.ToTensor(),
                                     ])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load('fruitmodel.pth')
model.eval()

In [ ]:
def predict_image(image):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)

    input = Variable(image_tensor)
    input = input.to(device)
    
    output = model(input)
    index = output.data.cpu().numpy().argmax()
    return index

In [ ]:
def get_random_images(num):
    data = datasets.ImageFolder(data_dir, transform=test_transforms)
    classes = data.classes

    indices = list(range(len(data)))
    np.random.shuffle(indices)
    idx = indices[:num]

    from torch.utils.data.sampler import SubsetRandomSampler
    sampler = SubsetRandomSampler(idx)
    loader = torch.utils.data.DataLoader(data, 
                   sampler=sampler, batch_size=num)
    
    dataiter = iter(loader)
    images, labels = dataiter.next()
    return images, labels

In [ ]:
from torch.autograd import Variable
classes = datasets.ImageFolder(data_dir, transform=test_transforms).classes

# convert to PIL img
to_pil = transforms.ToPILImage()
images, labels = get_random_images(5)
fig = plt.figure(figsize=(10,10))

for ii in range(len(images)):
    image = to_pil(images[ii])
    index = predict_image(image)

    sub = fig.add_subplot(1, len(images), ii+1)
    res = int(labels[ii]) == index

    sub.set_title(str(classes[index]) + ":" + str(res), {'color': 'w'})
    plt.axis('off')
    plt.imshow(image)
plt.show()

In [ ]:
from PIL import Image
img = Image.open('apple.jpg')
img

In [ ]:
print(classes[predict_image(img)])

## Hosting On Colab

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip

In [ ]:
!pip install streamlit

In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
  "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
!streamlit run app.py

## Git Management

In [ ]:
!git config --global user.email "ongzhizheng@gmail.com"
!git config --global user.name "Fogeinator"

In [ ]:
!git add app.py width_control.py fruitmodel.pth apple.jpg classified_apple.png loss.png random.png
!git commit -am "Add Imgs"

In [ ]:
!git push

In [ ]:
import streamlit as st

from PIL import Image
import os, shutil
import matplotlib.pyplot as plt
import numpy

from width_control import *

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.autograd import Variable

st.set_page_config(
  page_title="FoodRescue",
  page_icon=":seedling:",
  layout="centered",
  initial_sidebar_state="collapsed",
)
select_block_container_style()

# -------------------- Header --------------------

st.markdown('# FoodRescue - Rescuing your Food.')



# -------------------- Helper Functions --------------------

data_dir = 'data/dataset/test'

test_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                      transforms.ToTensor(),
                                    ])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load('fruitmodel.pth')
model.eval()

classes = datasets.ImageFolder(data_dir, transform=test_transforms).classes

@st.cache
def predict_image(image):
  image_tensor = test_transforms(image).float()
  image_tensor = image_tensor.unsqueeze_(0)
  input = Variable(image_tensor)
  input = input.to(device)
  output = model(input)
  index = output.data.cpu().numpy().argmax()
  return index

@st.cache
def get_random_images(num):
  data = datasets.ImageFolder(data_dir, transform=test_transforms)
  classes = data.classes
  indices = list(range(len(data)))
  np.random.shuffle(indices)
  idx = indices[:num]
  from torch.utils.data.sampler import SubsetRandomSampler
  sampler = SubsetRandomSampler(idx)
  loader = torch.utils.data.DataLoader(data, 
              sampler=sampler, batch_size=num)
  dataiter = iter(loader)
  images, labels = dataiter.next()
  return images, labels


# -------------------- Upload --------------------

st.markdown('<hr>', unsafe_allow_html=True)

st.markdown('## Is your apple/orange/banana spoilt?')
st.markdown('## 🍎🍊🍌')

img = None
uploaded_file = st.file_uploader('')

if uploaded_file is not None:
  img = Image.open(uploaded_file)
  # model only support 3 color channel image
  if img.mode in ("RGBA", "P"): img = img.convert("RGB")
  st.success('Uploaded!')

# -------------------- Display --------------------

img_display = st.empty()
col_width = False

if img:
  pred = classes[predict_image(img)]
  img_display.image(img, use_column_width=col_width)
  st.markdown('# {}'.format(pred))




